In [1]:
import sqlalchemy as db

In [2]:
engine = db.create_engine('sqlite:///mydb6.db')

In [3]:
conn = engine.connect()

In [4]:
metadata = db.MetaData()

In [5]:
cars = db.Table(
    'Car', metadata,
    db.Column('car_id', db.Integer, primary_key=True),
    db.Column('car_name', db.Text,),
    db.Column('car_country', db.Text),
    db.Column('car_mileage', db.Integer),
    db.Column('car_price', db.Integer)
)
cars

Table('Car', MetaData(), Column('car_id', Integer(), table=<Car>, primary_key=True, nullable=False), Column('car_name', Text(), table=<Car>), Column('car_country', Text(), table=<Car>), Column('car_mileage', Integer(), table=<Car>), Column('car_price', Integer(), table=<Car>), schema=None)

In [6]:
countrys = db.Table(
    'Country', metadata,
    db.Column('country_id', db.Integer, primary_key=True),
    db.Column('country_name', db.Text,),
    db.Column('country_capital', db.Text),
)
countrys

Table('Country', MetaData(), Column('country_id', Integer(), table=<Country>, primary_key=True, nullable=False), Column('country_name', Text(), table=<Country>), Column('country_capital', Text(), table=<Country>), schema=None)

In [7]:
metadata.create_all(engine)

In [8]:
insertion = cars.insert().values([
    {'car_name': 'Audi', 'car_country': 'Germany', 'car_mileage': 10000, 'car_price': 35500},
    {'car_name': 'BMW', 'car_country': 'Germany', 'car_mileage': 15600, 'car_price': 24500},
    {'car_name': 'Mercedes', 'car_country': 'Germany', 'car_mileage': 4000, 'car_price': 55500}

])

In [9]:
conn.execute(insertion)

In [10]:
select_all_query = db.select(cars)
select_result = conn.execute(select_all_query)
select_result

In [11]:
select_result.fetchall()

[(1, 'Audi', 'Germany', 10000, 35500),
 (2, 'BMW', 'Germany', 15600, 24500),
 (3, 'Mercedes', 'Germany', 4000, 55500)]

In [12]:
insertion2 = cars.insert().values([
    {'car_name': 'Lada', 'car_country': 'Russia', 'car_mileage': 5000, 'car_price': 20000}
])
conn.execute(insertion2)
select_all_query = db.select(cars)
select_result = conn.execute(select_all_query)
select_result.fetchall()

[(1, 'Audi', 'Germany', 10000, 35500),
 (2, 'BMW', 'Germany', 15600, 24500),
 (3, 'Mercedes', 'Germany', 4000, 55500),
 (4, 'Lada', 'Russia', 5000, 20000)]

In [13]:
select_where = db.select(cars).where(cars.columns.car_mileage < 12000)
select_result = conn.execute(select_where)
select_result.fetchall()

[(1, 'Audi', 'Germany', 10000, 35500),
 (3, 'Mercedes', 'Germany', 4000, 55500),
 (4, 'Lada', 'Russia', 5000, 20000)]

In [14]:
insertion3 = cars.insert().values([
    {'car_name': 'Lada', 'car_country': 'Russia', 'car_mileage': 7000, 'car_price': 15000},
    {'car_name': 'Lada', 'car_country': 'Russia', 'car_mileage': 8000, 'car_price': 17000},
    {'car_name': 'Mercedes', 'car_country': 'Germany', 'car_mileage': 8000, 'car_price': 75500}
])
conn.execute(insertion3)
select_all_query = db.select(cars)
select_result = conn.execute(select_all_query)
select_result.fetchall()

[(1, 'Audi', 'Germany', 10000, 35500),
 (2, 'BMW', 'Germany', 15600, 24500),
 (3, 'Mercedes', 'Germany', 4000, 55500),
 (4, 'Lada', 'Russia', 5000, 20000),
 (5, 'Lada', 'Russia', 7000, 15000),
 (6, 'Lada', 'Russia', 8000, 17000),
 (7, 'Mercedes', 'Germany', 8000, 75500)]

In [15]:
select_where = db.select(cars).where(cars.columns.car_mileage < 12000)
select_result = conn.execute(select_where)
select_result.fetchall()

[(1, 'Audi', 'Germany', 10000, 35500),
 (3, 'Mercedes', 'Germany', 4000, 55500),
 (4, 'Lada', 'Russia', 5000, 20000),
 (5, 'Lada', 'Russia', 7000, 15000),
 (6, 'Lada', 'Russia', 8000, 17000),
 (7, 'Mercedes', 'Germany', 8000, 75500)]

In [16]:
from sqlalchemy import func, desc, delete

In [17]:
select_where = db.select(cars.columns.car_name, func.sum(cars.columns.car_price)).group_by(cars.columns.car_name)
select_result = conn.execute(select_where)
select_result.fetchall()

[('Audi', 35500), ('BMW', 24500), ('Lada', 52000), ('Mercedes', 131000)]

In [18]:
select_order = db.select(cars.columns.car_name, func.sum(cars.columns.car_price).label('sum_price')).group_by(cars.columns.car_name).order_by(desc(cars.columns.car_name))
select_result = conn.execute(select_order)
select_result.fetchall()

[('Mercedes', 131000), ('Lada', 52000), ('BMW', 24500), ('Audi', 35500)]

In [19]:
print(select_order)

SELECT "Car".car_name, sum("Car".car_price) AS sum_price 
FROM "Car" GROUP BY "Car".car_name ORDER BY "Car".car_name DESC


In [20]:
insertion_c = countrys.insert().values([
    {'country_name': 'Germany', 'country_capital': 'Berlin'},
    {'country_name': 'Russia', 'country_capital': 'Moscow'}
])
insertion_c

In [21]:
conn.execute(insertion_c)
select_all_query = db.select(countrys)
select_result = conn.execute(select_all_query)
select_result.fetchall()

[(1, 'Germany', 'Berlin'), (2, 'Russia', 'Moscow')]

In [22]:
dele = db.delete(countrys).where(countrys.columns.country_id > 2)
conn.execute(dele)

In [23]:
select_all_query = db.select(countrys)
select_result = conn.execute(select_all_query)
select_result.fetchall()

[(1, 'Germany', 'Berlin'), (2, 'Russia', 'Moscow')]

In [24]:
print(dele)

DELETE FROM "Country" WHERE "Country".country_id > :country_id_1


In [25]:
join_1 = db.select(cars.columns.car_name, countrys.columns.country_capital).join_from(countrys, cars, cars.columns.car_country == countrys.columns.country_name)

In [26]:
print(join_1)

SELECT "Car".car_name, "Country".country_capital 
FROM "Country" JOIN "Car" ON "Car".car_country = "Country".country_name


In [27]:
last_res = conn.execute(join_1)

In [28]:
last_res.fetchall()

[('Audi', 'Berlin'),
 ('BMW', 'Berlin'),
 ('Mercedes', 'Berlin'),
 ('Mercedes', 'Berlin'),
 ('Lada', 'Moscow'),
 ('Lada', 'Moscow'),
 ('Lada', 'Moscow')]

In [29]:
subq = (db.select(cars).where(cars.columns.car_mileage < 12000).cte())
stmt = db.select(subq.columns.car_name, countrys.columns.country_capital).join_from(countrys, subq, subq.columns.car_country == countrys.columns.country_name)
print(stmt)

WITH anon_1 AS 
(SELECT "Car".car_id AS car_id, "Car".car_name AS car_name, "Car".car_country AS car_country, "Car".car_mileage AS car_mileage, "Car".car_price AS car_price 
FROM "Car" 
WHERE "Car".car_mileage < :car_mileage_1)
 SELECT anon_1.car_name, "Country".country_capital 
FROM "Country" JOIN anon_1 ON anon_1.car_country = "Country".country_name


In [30]:
last_res = conn.execute(stmt)
last_res.fetchall()

[('Audi', 'Berlin'),
 ('Mercedes', 'Berlin'),
 ('Lada', 'Moscow'),
 ('Lada', 'Moscow'),
 ('Lada', 'Moscow'),
 ('Mercedes', 'Berlin')]